In [1]:
%load_ext autoreload
%autoreload 2

In [137]:
import numpy as np
from tqdm import tqdm

from jaccard import *

In [3]:
np.set_printoptions(precision=4, suppress=True, floatmode='fixed')

In [4]:
m = 3
n = 5
avg_votes = 2
projects_to_found = 2

In [5]:
costs = np.ones(m) / projects_to_found - 1e-6
def cost(center: np.ndarray) -> float:
    return (center * costs).sum()
print(costs)

[0.5000 0.5000 0.5000]


In [ ]:
def dist_to_centers(V: np.ndarray, centers: np.ndarray) -> tuple[np.ndarray, np.ndarray]:
    d = jaccard(V, centers)
    return d.min(axis=1), d.argmin(axis=1)
def remove_bad_centers(V: np.ndarray, centers: np.ndarray) -> np.ndarray:
    while True:
        _, assignment = dist_to_centers(V, centers)
        efficiency = [
            ((n2 / n) / cost(centers[c]), c)
            for c, n2 in zip(*np.unique(assignment, return_counts=True))
        ]
        for i in range(len(centers)):
            if i not in {c for _, c in efficiency}:
                efficiency.append((0, i))
        eff, c = min(efficiency)
        if eff < 1.0:
            centers = np.delete(centers, c, axis=0)
        else:
            break
    return centers

In [227]:
linspace = np.linspace(0, 1, 10 + 1)
def find_candidates(lst: list[float]):
    if len(lst) == m:
        yield lst
        return
    for x in linspace:
        lst2 = lst + [x]
        yield from find_candidates(lst2)
candidates = np.array(list(find_candidates([]))[1:])
print(candidates, len(candidates))

def is_blocking_coalition(V, c, old) -> bool:
    new = jaccard(V, c.reshape(1, m)).reshape(-1)
    return np.sum(new + 1e-15 < old) / n >= cost(c)
def find_blocking_coalition(V: np.ndarray, centers: np.ndarray) -> np.ndarray | None:
    old, _ = dist_to_centers(V, centers)
    for v in candidates:
        if is_blocking_coalition(V, v, old):
            return v
    return None

[[0.0000 0.0000 0.1000]
 [0.0000 0.0000 0.2000]
 [0.0000 0.0000 0.3000]
 ...
 [1.0000 1.0000 0.8000]
 [1.0000 1.0000 0.9000]
 [1.0000 1.0000 1.0000]] 1330


In [228]:
def converges(V: np.ndarray, C: np.ndarray) -> bool:
    history = set(tuple(map(tuple, C)))
    for _ in range(1000):
        c = find_blocking_coalition(V, C)
        if c is not None:
            C = np.append(C, c.reshape(1, m), axis=0)
            #C = remove_bad_centers(V, C)
            C = C[np.unique(dist_to_centers(V, C)[1])]
            h = tuple(map(tuple, C))
            if h in history:
                return False
            history.add(h)
        else:
            break
    else:
        raise RuntimeError('Too few iterations')
    return True

In [230]:
for i in tqdm(range(10000)):
    V = np.random.choice([0, 1], (10, m), p=[1 - avg_votes / m, avg_votes / m])
    C = remove_bad_centers(V, np.eye(m))
    if not converges(V, C):
        print("Does not converge!")
        print(V)
        print(C)
        break

  0%|          | 0/10000 [00:00<?, ?it/s]

  2%|▏         | 186/10000 [00:18<16:38,  9.83it/s]


KeyboardInterrupt: 

In [199]:
#V = np.random.choice([0, 1], (n, m), p=[1 - avg_votes / m, avg_votes / m])
V = np.array([
    [1,1,0],
    [1,1,0],
    [1,0,1],
    [1,0,1],
    [1,0,0],
])
C = remove_bad_centers(V, np.eye(m))

history = set(tuple(map(tuple, C)))
blocking_coalitions = []

for i in range(1000):
    c = find_blocking_coalition(V, C)
    if c is not None:
        blocking_coalitions.append(c)
        C = np.append(C, c.reshape(1, m), axis=0)
        C = remove_bad_centers(V, C)
        #print(f"{i=:<4}  centers={len(C):<2}  blocking coalition: {c}")
        print(f"{i=:<4}")
        print(f"blocking coalition: {c}")
        print(C)
        print()
        h = tuple(map(tuple, C))
        if h in history:
            print("Repetition!")
            break
        history.add(h)
    else:
        break
else:
    print(f"Did not converge!")
dist, assignment = dist_to_centers(V, C)
print(V)
print(assignment.reshape(-1, 1))
print(C)

i=0   
blocking coalition: [1.0000 0.2000 0.2000]
[[1.0000 0.2000 0.2000]]

i=1   
blocking coalition: [1.0000 0.0000 0.2000]
[[1.0000 0.0000 0.2000]]

i=2   
blocking coalition: [1.0000 0.0000 0.0000]
[[1.0000 0.0000 0.0000]]

i=3   
blocking coalition: [1.0000 0.2000 0.2000]
[[1.0000 0.2000 0.2000]]

Repetition!
[[1 1 0]
 [1 1 0]
 [1 0 1]
 [1 0 1]
 [1 0 0]]
[[0]
 [0]
 [0]
 [0]
 [0]]
[[1.0000 0.2000 0.2000]]


In [154]:
blocking_coalitions

[array([1.0000, 0.2000, 0.2000]),
 array([1.0000, 0.0000, 0.2000]),
 array([1.0000, 0.0000, 0.0000]),
 array([1.0000, 0.2000, 0.2000])]

In [ ]:
V = np.array([
    [1, 1, 0],
    [1, 1, 0],
    [1, 0, 1],
    [1, 0, 1],
    [1, 0, 0],
])
C = remove_bad_centers(V, np.eye(m))
print(V)
print(C)

history = set(tuple(map(tuple, C)))

coalitions = [
 np.array([1.0, 0.2, 0.2]),
 np.array([1.0, 0.0, 0.2]),
 np.array([1.0, 0.0, 0.0]),
 np.array([1.0, 0.2, 0.2])
]

for i, c in enumerate(coalitions):
    dist, _ = dist_to_centers(V, C)
    assert is_blocking_coalition(V, c, dist), f"c={c}\nC={C}"
    if c is not None:
        C = np.append(C, c.reshape(1, m), axis=0)
        C = remove_bad_centers(V, C)
        #print(f"{i=:<4}  centers={len(C):<2}  blocking coalition: {c}")
        print(f"{i=:<4}")
        print(c)
        print(C)
        print()
        h = tuple(map(tuple, C))
        if h in history:
            print("Repetition!")
            break
        history.add(h)
    else:
        break
else:
    print(f"Did not converge!")

[[1 1 0]
 [1 1 0]
 [1 0 1]
 [1 0 1]
 [1 0 0]]
[[1.0000 0.0000 0.0000]]


AssertionError: c=[1.0000 0.0000 0.2000]
C=[[1.0000 0.0000 0.0000]]

In [224]:
V = np.array([[1, 0]])
C = np.array([[1, 0], [0, 1]])
C[np.unique(dist_to_centers(V, C)[1])]

array([[1, 0]])